## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1.
Обязательная часть Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: - - - <текст статьи>


In [45]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
#time.sleep(0.2)

In [12]:
kom_news = pd.DataFrame()
KEYWORDS = ['python', 'парсинг']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
#soup

posts = soup.find_all('article', class_='post')
posts

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    #print(hubs)    
    for hub in hubs:
        #if any([hub in desired for desired in KEYWORDS]):
        hub_lower = hub.text.lower()
        #print(hub_lower)
        if any([hub_lower in desired for desired in KEYWORDS]):
            title_element = post.find('a', class_='post__title_link')
            row = {'title': title_element, 'link': title_element.attrs.get('href')}
            kom_news = pd.concat([kom_news, pd.DataFrame([row])])
            print(title_element.text, title_element.attrs.get('href'))
            
        

Serverless телеграм бот с использованием Яндекс облака https://habr.com/ru/post/522126/
Отношение один к одному: связывание модели пользователя с кастомной моделью профиля в Django https://habr.com/ru/company/otus/blog/522106/
7 бесплатных книг, которые следует прочитать каждому дата-сайентисту https://habr.com/ru/company/skillfactory/blog/521834/


## Задание 2.
### Обязательная часть 
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology' TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [66]:
import json 

In [95]:
#MAIL = ['yxx@mail.ru']
MAIL = ['xxx@x.ru', 'yyy@y.com']
#URL = 'https://in.hotjar.com/api/v2/client/sites/470805/visit-data?sv=7'
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
#URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data?'
#params = {'status': 'ok', 'value':{'already_subscribed': 'false'},'email': 'ada@severtm.ru'}

params = {
    'emailAddresses': MAIL
}

headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
}

r = requests.post(URL, data = json.dumps(params), headers = headers)
#r = requests.post(URL, params, headers=headers)
#r = requests.post(URL, params)
r.json()


df = pd.DataFrame(r.json()['breaches'])
df = df.transpose().reset_index()
df = df.rename(columns={'index': 'breach'})
df

,breach,site,recordsCount,description,publishDate
0,16613,verifications.io,677914246,Big data e-mail verification platform verifica...,2019-03-28T00:00:00Z
1,17609,vk.com,42542545,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
2,16488,www.dangdang.com,4847249,"This is a list of email addresses only, and as...",2019-02-21T00:00:00Z
3,17110,azcentral.com,705538,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z
4,17670,wishbone.io,37475483,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z
5,13094,myheritage.com,110042353,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z
6,13254,creocommunity.com,479182,"At an unconfirmed date, Creo Community's user ...",2017-12-01T00:00:00Z
7,16768,canva.com,137434012,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z
8,3176,parapa.mail.ru,5029003,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z
9,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z
